In [1]:
from pathlib import Path
import pandas as pd

from utils import *
from dataloaders import *
from data_preparation import *
from allResNets import *
from MobileNetV3 import *
from engine import *
from efficientNet import *
from data_transforms import *

change_to_disk()
data_dir = Path("clean_split_1k/")
models_path = Path('Models_1/')
check_dir(models_path)

In [2]:
image_shape = (1080, 1920)
resize_factor = 0.3
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

BATCH_SIZE = 16
train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders(data_dir, data_transform, batch_size = BATCH_SIZE)

In [3]:
NUM_EPOCHS = 100
lr = 0.001
device = get_device()

resnet18 = ResNet18()
resnet50 = ResNet50()
mobnet_sml = MobileNetSmall()
mobnet_lrg = MobileNetLarge()
effnetb0 = EfficientNetB0()

loss_fn = nn.CrossEntropyLoss()

In [8]:
model_name = "ResNet18"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet18
resnet18.to(device)
nadam_optim = torch.optim.NAdam(params=resnet18.parameters(), lr=lr)
train_resnet18_results, train_time_resnet18 = train(resnet18, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0144 | train_acc: 0.5623 | validation_loss: 0.7825 | validation_acc: 0.6543
Epoch: 2 | train_loss: 0.6989 | train_acc: 0.6873 | validation_loss: 0.7271 | validation_acc: 0.6836
Epoch: 3 | train_loss: 0.6328 | train_acc: 0.7225 | validation_loss: 1.9663 | validation_acc: 0.4609
Epoch: 4 | train_loss: 0.5559 | train_acc: 0.7585 | validation_loss: 3.1090 | validation_acc: 0.4551
Epoch: 5 | train_loss: 0.4731 | train_acc: 0.7985 | validation_loss: 0.8597 | validation_acc: 0.6523
Epoch: 6 | train_loss: 0.4698 | train_acc: 0.7957 | validation_loss: 0.6161 | validation_acc: 0.7305
Epoch: 7 | train_loss: 0.3937 | train_acc: 0.8375 | validation_loss: 1.9131 | validation_acc: 0.5703
Epoch: 8 | train_loss: 0.3703 | train_acc: 0.8485 | validation_loss: 0.7428 | validation_acc: 0.7422
Epoch: 9 | train_loss: 0.3264 | train_acc: 0.8702 | validation_loss: 21.4177 | validation_acc: 0.3516
Epoch: 10 | train_loss: 0.3325 | train_acc: 0.8695 | validation_loss: 0.5615 | validation_

In [9]:
eval_resnet18_results = eval_model(resnet18, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
resnet18_results = [train_resnet18_results, train_time_resnet18, eval_resnet18_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, resnet18, resnet18_results)

  0%|          | 0/32 [00:02<?, ?it/s]

In [5]:
model_name = "ResNet50"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet50
resnet50.to(device)
nadam_optim = torch.optim.NAdam(params=resnet50.parameters(), lr=lr)
train_resnet50_results, train_time_resnet50 = train(resnet50, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

Models_1\ResNet50 does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.5097 | train_acc: 0.3332 | validation_loss: 1.9258 | validation_acc: 0.3730
Epoch: 2 | train_loss: 1.1713 | train_acc: 0.4778 | validation_loss: 2.5119 | validation_acc: 0.3125
Epoch: 3 | train_loss: 1.0093 | train_acc: 0.5567 | validation_loss: 7.7010 | validation_acc: 0.3320
Epoch: 4 | train_loss: 0.8853 | train_acc: 0.5905 | validation_loss: 8.5414 | validation_acc: 0.1973
Epoch: 5 | train_loss: 0.8283 | train_acc: 0.6252 | validation_loss: 7.2213 | validation_acc: 0.3125
Epoch: 6 | train_loss: 0.8092 | train_acc: 0.6295 | validation_loss: 9.0424 | validation_acc: 0.2480
Epoch: 7 | train_loss: 0.7898 | train_acc: 0.6388 | validation_loss: 1.4425 | validation_acc: 0.5488
Epoch: 8 | train_loss: 0.7186 | train_acc: 0.6750 | validation_loss: 13.0369 | validation_acc: 0.3594
Epoch: 9 | train_loss: 0.7510 | train_acc: 0.6667 | validation_loss: 1.7984 | validation_acc: 0.5293
Epoch: 10 | train_loss: 0.6823 | train_acc: 0.6890 | validation_loss: 1.6425 | validation_

In [6]:
eval_resnet50_results = eval_model(resnet50, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
resnet50_results = [train_resnet50_results, train_time_resnet50, eval_resnet50_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, resnet50, resnet50_results)

  0%|          | 0/32 [00:02<?, ?it/s]

In [7]:
model_name = "MobileNetSmall"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train MobileNetV3 Small
mobnet_sml.to(device)
nadam_optim = torch.optim.NAdam(params=mobnet_sml.parameters(), lr=lr)
train_mobnet_sml_results, train_time_mobnet_sml = train(mobnet_sml, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.2775 | train_acc: 0.4170 | validation_loss: 0.9305 | validation_acc: 0.5703
Epoch: 2 | train_loss: 0.9190 | train_acc: 0.5793 | validation_loss: 0.6777 | validation_acc: 0.6387
Epoch: 3 | train_loss: 0.6605 | train_acc: 0.7147 | validation_loss: 0.7538 | validation_acc: 0.6816
Epoch: 4 | train_loss: 0.5553 | train_acc: 0.7688 | validation_loss: 0.8607 | validation_acc: 0.6602
Epoch: 5 | train_loss: 0.4412 | train_acc: 0.8193 | validation_loss: 0.9628 | validation_acc: 0.6504
Epoch: 6 | train_loss: 0.3497 | train_acc: 0.8640 | validation_loss: 0.6014 | validation_acc: 0.7930
Epoch: 7 | train_loss: 0.3202 | train_acc: 0.8722 | validation_loss: 0.4549 | validation_acc: 0.8398
Epoch: 8 | train_loss: 0.2179 | train_acc: 0.9243 | validation_loss: 0.8328 | validation_acc: 0.7793
Epoch: 9 | train_loss: 0.2017 | train_acc: 0.9303 | validation_loss: 0.6922 | validation_acc: 0.7891
Epoch: 10 | train_loss: 0.1843 | train_acc: 0.9365 | validation_loss: 0.6192 | validation_a

In [8]:
eval_mobnet_sml_results = eval_model(mobnet_sml, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
mobnet_sml_results = [train_mobnet_sml_results, train_time_mobnet_sml, eval_mobnet_sml_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, mobnet_sml, mobnet_sml_results)

  0%|          | 0/32 [00:02<?, ?it/s]

In [4]:
model_name = "MobileNetLarge"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train MobileNetV3 Large
mobnet_lrg.to(device)
nadam_optim = torch.optim.NAdam(params=mobnet_lrg.parameters(), lr=lr)
train_mobnet_lrg_results, train_time_mobnet_lrg = train(mobnet_lrg, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0643 | train_acc: 0.5210 | validation_loss: 0.6741 | validation_acc: 0.7012
Epoch: 2 | train_loss: 0.7024 | train_acc: 0.6797 | validation_loss: 0.7943 | validation_acc: 0.6758
Epoch: 3 | train_loss: 0.5379 | train_acc: 0.7860 | validation_loss: 0.9604 | validation_acc: 0.5918
Epoch: 4 | train_loss: 0.4308 | train_acc: 0.8270 | validation_loss: 1.0418 | validation_acc: 0.6777
Epoch: 5 | train_loss: 0.3389 | train_acc: 0.8675 | validation_loss: 1.2787 | validation_acc: 0.6484
Epoch: 6 | train_loss: 0.3037 | train_acc: 0.8915 | validation_loss: 1.1228 | validation_acc: 0.6875
Epoch: 7 | train_loss: 0.2537 | train_acc: 0.9100 | validation_loss: 1.1446 | validation_acc: 0.6797
Epoch: 8 | train_loss: 0.2398 | train_acc: 0.9107 | validation_loss: 0.7272 | validation_acc: 0.7734
Epoch: 9 | train_loss: 0.2124 | train_acc: 0.9195 | validation_loss: 1.4009 | validation_acc: 0.7129
Epoch: 10 | train_loss: 0.2002 | train_acc: 0.9285 | validation_loss: 0.8661 | validation_a

In [5]:
eval_mobnet_lrg_results = eval_model(mobnet_lrg, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
mobnet_lrg_results = [train_mobnet_lrg_results, train_time_mobnet_lrg, eval_mobnet_lrg_results]
final_save(model_folder_path, model_name, NUM_EPOCHS, mobnet_lrg, mobnet_lrg_results)

  0%|          | 0/32 [00:01<?, ?it/s]

In [6]:
model_name = "EfficientNetB0"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train EfficientNet B0
effnetb0.to(device)
nadam_optim = torch.optim.NAdam(params=effnetb0.parameters(), lr=lr)
train_effnetb0_results, train_time_effnetb0 = train(effnetb0, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

Models_1\EfficientNetB0 does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1400 | train_acc: 0.4928 | validation_loss: 0.6974 | validation_acc: 0.7070
Epoch: 2 | train_loss: 0.4597 | train_acc: 0.8137 | validation_loss: 0.5444 | validation_acc: 0.7656
Epoch: 3 | train_loss: 0.2851 | train_acc: 0.8925 | validation_loss: 0.4863 | validation_acc: 0.8242
Epoch: 4 | train_loss: 0.2174 | train_acc: 0.9243 | validation_loss: 0.4243 | validation_acc: 0.8496
Epoch: 5 | train_loss: 0.1798 | train_acc: 0.9347 | validation_loss: 0.7281 | validation_acc: 0.7891
Epoch: 6 | train_loss: 0.1499 | train_acc: 0.9500 | validation_loss: 0.5425 | validation_acc: 0.8652
Epoch: 7 | train_loss: 0.1421 | train_acc: 0.9457 | validation_loss: 0.6720 | validation_acc: 0.8535
Epoch: 8 | train_loss: 0.1270 | train_acc: 0.9557 | validation_loss: 0.6613 | validation_acc: 0.8633
Epoch: 9 | train_loss: 0.1168 | train_acc: 0.9607 | validation_loss: 0.4938 | validation_acc: 0.8750
Epoch: 10 | train_loss: 0.1034 | train_acc: 0.9645 | validation_loss: 0.6537 | validation_a

In [7]:
eval_effnetb0_results = eval_model(effnetb0, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
effnetb0_results = [train_effnetb0_results, train_time_effnetb0, eval_effnetb0_results]
final_save(model_folder_path, model_name, NUM_EPOCHS, effnetb0, effnetb0_results)

  0%|          | 0/32 [00:01<?, ?it/s]